In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Get Device for Training

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


### Define the Class

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


### Model Layers

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


`nn.Flatten`

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


`nn.Linear`

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


`nn.ReLU`

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1395, -0.1861,  0.0917, -0.3379, -0.1056, -0.1757,  0.1938,  0.1319,
          0.0762, -0.8590,  0.0328,  0.4244, -0.1436,  0.3695,  0.0644, -0.1297,
         -0.1263, -0.4510,  0.3533, -0.1444],
        [ 0.0679, -0.1490,  0.2017, -0.5774, -0.2181, -0.0091, -0.0743,  0.2819,
          0.1406, -0.8443,  0.2997, -0.1973, -0.0668,  0.4386, -0.0699,  0.3192,
         -0.0962, -0.1145, -0.1431, -0.1176],
        [ 0.1312, -0.1482,  0.0901, -0.2705, -0.2110,  0.0185, -0.4501, -0.0254,
          0.0478, -0.8993, -0.0784,  0.2915, -0.2258,  0.6114, -0.0803,  0.2670,
         -0.0286, -0.2148,  0.4402, -0.4366]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0917, 0.0000, 0.0000, 0.0000, 0.1938, 0.1319, 0.0762,
         0.0000, 0.0328, 0.4244, 0.0000, 0.3695, 0.0644, 0.0000, 0.0000, 0.0000,
         0.3533, 0.0000],
        [0.0679, 0.0000, 0.2017, 0.0000, 0.0000, 0.0000, 0.0000, 0.2819, 0.1406,
         0.0000, 0.2997, 0.0000, 0.0000, 0.4386, 0.00

`nn.Sequantial`

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

`nn.Softmax`

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### Model Parameters

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0213,  0.0060, -0.0194,  ..., -0.0285, -0.0297, -0.0337],
        [-0.0291, -0.0249, -0.0277,  ..., -0.0105, -0.0140,  0.0168]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0306, 0.0302], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0360,  0.0043, -0.0020,  ...,  0.0010,  0.0259,  0.0159],
        [ 0.0384, -0.0147, -0.0009,  ..., -0.0372, -0.0432, -0.0131]],
       device='cuda:0', grad_fn=<Slic

## Optimizing Model Parameters

### Prerequisite Code

In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████████████████████████████████| 26.4M/26.4M [00:03<00:00, 6.75MB/s]
100%|███████████████████████████████████████| 29.5k/29.5k [00:00<00:00, 103kB/s]
100%|██████████████████████████████████████| 4.42M/4.42M [00:02<00:00, 1.92MB/s]
100%|██████████████████████████████████████| 5.15k/5.15k [00:00<00:00, 19.5MB/s]


### Hyperparameters

In [14]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### Optimization Loop

#### Loss Function

In [15]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

#### Optimizer

In [16]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Full Implementation

In [17]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.312067  [   64/60000]
loss: 2.293219  [ 6464/60000]
loss: 2.270792  [12864/60000]
loss: 2.267136  [19264/60000]
loss: 2.239344  [25664/60000]
loss: 2.231098  [32064/60000]
loss: 2.238086  [38464/60000]
loss: 2.199864  [44864/60000]
loss: 2.199477  [51264/60000]
loss: 2.173437  [57664/60000]
Test Error: 
 Accuracy: 43.8%, Avg loss: 2.159869 

Epoch 2
-------------------------------
loss: 2.174911  [   64/60000]
loss: 2.162626  [ 6464/60000]
loss: 2.099812  [12864/60000]
loss: 2.123848  [19264/60000]
loss: 2.047157  [25664/60000]
loss: 2.009048  [32064/60000]
loss: 2.048757  [38464/60000]
loss: 1.953727  [44864/60000]
loss: 1.973747  [51264/60000]
loss: 1.911024  [57664/60000]
Test Error: 
 Accuracy: 45.3%, Avg loss: 1.893232 

Epoch 3
-------------------------------
loss: 1.929249  [   64/60000]
loss: 1.902736  [ 6464/60000]
loss: 1.774734  [12864/60000]
loss: 1.835697  [19264/60000]
loss: 1.693219  [25664/60000]
loss: 1.659599  [32064/600